In [ ]:
import os
import random
import pandas as pd
from google.colab import files
from pathlib import Path
import zipfile
import re

In [ ]:
# === PARAMETRY ===
PROJECT_NAME = "project"
SEED = 1
random.seed(SEED)


In [ ]:
# === UPLOAD ZIP ===
print("➡️ Załaduj folder ZIP zawierający pliki .nii/.nii.gz")
uploaded = files.upload()

➡️ Załaduj folder ZIP zawierający pliki .nii/.nii.gz


Saving wiek.zip to wiek (1).zip


In [ ]:
# === ROZPAKOWANIE ZIPA ===
for zip_name in uploaded.keys():
    zip_path = f"/content/{zip_name}"
    extract_path = "/content/nifti_data"
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

# === WYSZUKIWANIE PLIKÓW .nii/.nii.gz ===
nii_paths = list(Path(extract_path).rglob("*.nii")) + list(Path(extract_path).rglob("*.nii.gz"))
nii_paths = sorted(nii_paths)  # Ustalona kolejność dla replikacji
total_files = len(nii_paths)

# === PODZIAŁ NA PARTITION 80/10/10 ===
train_end = int(0.7 * total_files)
dev_end = train_end + int(0.15 * total_files)

partition_list = ['train'] * train_end + ['dev'] * (dev_end - train_end) + ['test'] * (total_files - dev_end)
random.shuffle(partition_list)  # Miesza dane przed przypisaniem

In [ ]:
# === TWORZENIE CSV ===
rows = []

for i, (path, partition) in enumerate(zip(nii_paths, partition_list)):
    filename = path.name
    abs_path = str(path.resolve())

    # UID: 3 cyfry po "IXI"
    match = re.search(r"IXI(\d{3})", filename)
    if not match:
        print(f"⚠️ Nie udało się wyciągnąć UID z: {filename}")
        continue
    uid = match.group(1)

    rows.append({
        "indx": f"img_{i}",  # malejąco
        "Project": PROJECT_NAME,
        "uid": uid,
        "path_registered": abs_path,
        "age_at_scan": None,
        "partition": partition
    })

df = pd.DataFrame(rows)

In [ ]:
df

,indx,Project,uid,path_registered,age_at_scan,partition
0,img_0,project,064,/content/nifti_data/wiek/1-IXI064-G-T1-age_20....,None,train
1,img_1,project,070,/content/nifti_data/wiek/1-IXI070-G-T1-age_21....,None,dev
2,img_2,project,202,/content/nifti_data/wiek/1-IXI202-H-T1-age_20....,None,train
3,img_3,project,278,/content/nifti_data/wiek/1-IXI278-H-T1-age_20....,None,test
4,img_4,project,448,/content/nifti_data/wiek/2-IXI448-H-T1-age_80....,None,train
5,img_5,project,499,/content/nifti_data/wiek/2-IXI499-G-T1-age_82....,None,train
6,img_6,project,538,/content/nifti_data/wiek/2-IXI538-H-T1-age_82....,None,train
7,img_7,project,639,/content/nifti_data/wiek/2-IXI639-G-T1-age_86....,None,test


In [ ]:
# === UPLOAD CSV ===
print("➡️ Załaduj plik csv z wiekiem")
uploaded = files.upload()

➡️ Załaduj plik csv z wiekiem


Saving file_for_preprocessing_with_registered.csv to file_for_preprocessing_with_registered (1).csv


In [ ]:
df_age = pd.read_csv('file_for_preprocessing_with_registered.csv')


In [ ]:
df_age

,indx,Project,uid,age_at_scan,partition,path,path_registered
0,460,project,460,60,train,/Users/mlesniak/Documents/studia/wb/MRI/braina...,/Users/mlesniak/Documents/studia/wb/MRI/braina...
1,407,project,407,58,train,/Users/mlesniak/Documents/studia/wb/MRI/braina...,/Users/mlesniak/Documents/studia/wb/MRI/braina...
2,148,project,148,40,train,/Users/mlesniak/Documents/studia/wb/MRI/braina...,/Users/mlesniak/Documents/studia/wb/MRI/braina...
3,182,project,182,60,train,/Users/mlesniak/Documents/studia/wb/MRI/braina...,/Users/mlesniak/Documents/studia/wb/MRI/braina...
4,169,project,169,58,train,/Users/mlesniak/Documents/studia/wb/MRI/braina...,/Users/mlesniak/Documents/studia/wb/MRI/braina...
...,...,...,...,...,...,...,...
558,560,project,560,63,test,/Users/mlesniak/Documents/studia/wb/MRI/braina...,/Users/mlesniak/Documents/studia/wb/MRI/braina...
559,621,project,621,31,test,/Users/mlesniak/Documents/studia/wb/MRI/braina...,/Users/mlesniak/Documents/studia/wb/MRI/braina...
560,423,project,423,39,test,/Users/mlesniak/Documents/studia/wb/MRI/braina...,/Users/mlesniak/Documents/studia/wb/MRI/braina...
561,524,project,524,33,test,/Users/mlesniak/Documents/studia/wb/MRI/braina...,/Users/mlesniak/Documents/studia/wb/MRI/braina...


In [ ]:
# inputing age
# Upewnij się, że kolumny uid są stringami i z 0-paddingiem (np. '045', '122')
df['uid'] = df['uid'].astype(str).str.zfill(3)
df_age['uid'] = df_age['uid'].astype(str).str.zfill(3)

# Upewnij się, że df_age ma kolumny: 'uid', 'age_at_scan'
# Wypełnij brakujące wartości w df_main
df['age_at_scan'] = df['age_at_scan'].fillna(
    df['uid'].map(df_age.set_index('uid')['age_at_scan'])
)


<ipython-input-44-c86659bf84ff>:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['age_at_scan'] = df['age_at_scan'].fillna(


In [ ]:
df

,indx,Project,uid,path_registered,age_at_scan,partition
0,img_0,project,064,/content/nifti_data/wiek/1-IXI064-G-T1-age_20....,20,train
1,img_1,project,070,/content/nifti_data/wiek/1-IXI070-G-T1-age_21....,20,dev
2,img_2,project,202,/content/nifti_data/wiek/1-IXI202-H-T1-age_20....,20,train
3,img_3,project,278,/content/nifti_data/wiek/1-IXI278-H-T1-age_20....,20,test
4,img_4,project,448,/content/nifti_data/wiek/2-IXI448-H-T1-age_80....,79,train
5,img_5,project,499,/content/nifti_data/wiek/2-IXI499-G-T1-age_82....,82,train
6,img_6,project,538,/content/nifti_data/wiek/2-IXI538-H-T1-age_82....,81,train
7,img_7,project,639,/content/nifti_data/wiek/2-IXI639-G-T1-age_86....,86,test


In [ ]:
# === ZAPIS CSV ===
csv_path = "/content/input.csv"
df.to_csv(csv_path, index=False)

print("✅ Zapisano plik: input.csv")
files.download(csv_path)

✅ Zapisano plik: input.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>